In [ ]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import torch
from pathlib import Path

from lac.perception.segmentation import Segmentation
from lac.perception.depth import stereo_depth_from_segmentation, project_pixel_to_rover
from lac.control.controller import rock_avoidance_steering, ArcPlanner
from lac.utils.visualization import overlay_mask, overlay_stereo_rock_depths
from lac.utils.plotting import (
    plot_points_rover_frame,
    plot_path_rover_frame,
    plot_rocks_rover_frame,
)
from lac.utils.frames import invert_transform_mat
from lac.util import load_data, get_positions_from_poses
import lac.params as params

%load_ext autoreload
%autoreload 2

## Obstacle detection


In [ ]:
segmentation = Segmentation()

In [ ]:
# data_path = Path("../../output/NavAgent/map1_preset4_gtnav_steer")
data_path = Path("../../output/DataCollectionAgent/map1_preset9_rocks")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
i = 1154

left_image = Image.open(data_path / "FrontLeft" / f"{i}.png")
right_image = Image.open(data_path / "FrontRight" / f"{i}.png")

left_seg_masks, left_seg_full_mask = segmentation.segment_rocks(left_image.convert("RGB"))
right_seg_masks, right_seg_full_mask = segmentation.segment_rocks(right_image.convert("RGB"))

results = stereo_depth_from_segmentation(
    left_seg_masks, right_seg_masks, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_mask(np.array(left_image), left_seg_full_mask)
left_overlay = overlay_stereo_rock_depths(left_overlay, results)
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(left_overlay)
plt.axis("off")

In [ ]:
x, _, w, _ = cv.boundingRect(results[0]["left_mask"].astype(np.uint8))
width_x = w * results[0]["depth"] / params.FL_X
print(f"Width: {width_x} m")

In [ ]:
def extract_rock_radius(results, params):
    """
    Computes radii of detected rocks based on depth and bounding box width.

    Args:
        results (list): A list of dictionaries containing rock detection masks and depth values.
        params (object): An object containing camera parameters, specifically `FL_X` (focal length in pixels).

    Returns:
        list: A list of tuples [(x, y, radius), ...] representing the coordinates and estimated radius of each rock.
    """
    rock_data = []

    for result in results:
        if "left_mask" not in result or "depth" not in result:
            continue  # Skip if required keys are missing

        # Convert mask to uint8 and get bounding box
        mask_uint8 = result["left_mask"].astype(np.uint8)
        x, y, w, h = cv.boundingRect(mask_uint8)

        # Compute real-world width and radius using depth and focal length
        width_real = w * result["depth"] / params.FL_X
        radius_real = width_real / 2  # Approximate the radius

        rock_data.append(radius_real)
    return rock_data

In [ ]:
rock_points_rover_frame = []
rock_data = extract_rock_radius(results, params)


for rock in results:
    rock_points_rover_frame.append(
        project_pixel_to_rover(rock["left_centroid"], rock["depth"], "FrontLeft", cam_config)
    )

rock_points_rover_frame = np.array(rock_points_rover_frame)
print(rock_points_rover_frame)
plot_points_rover_frame(rock_points_rover_frame, color="red")
plot_rocks_rover_frame(rock_points_rover_frame, rock_data, color="blue")


## Dynamics characterization


In [ ]:
data_path = Path("../../output/DataCollectionAgent/map1_preset0_v0.2_w-0.5")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
# Transform poses from world to initial local rover frame
poses_local = [invert_transform_mat(initial_pose) @ pose for pose in poses]

positions_local = get_positions_from_poses(poses_local)
plot_path_rover_frame(positions_local, color="blue")

## Arc path planning


In [ ]:
planner = ArcPlanner()


arcs = planner.np_candidate_arcs

In [ ]:
print(rock_points_rover_frame)
test_rock_points_rover_frame = np.array([[0.5, 0.5], [0.4, -0.4], [0.2, 0.3]])

In [ ]:
test_rock_data = [0.07, 0.06, 0.05]

In [ ]:
# print(f"candidate arcs shape: {planner.np_candidate_arcs.shape}")
waypoint = np.array([0, 2])
current_pose = np.eye(4)
control, best_arc, waypoint_local = planner.plan_arc(
    waypoint, current_pose, test_rock_points_rover_frame, test_rock_data
)
# (v,w), waypoint_local = planner.plan_arc(waypoint, current_pose, test_rock_points_rover_frame, test_rock_data)
# print(waypoint_local)
fig = plot_rocks_rover_frame(
    test_rock_points_rover_frame, test_rock_data, waypoint=waypoint_local, color="red"
)
# fig = go.Figure()
for arc in arcs:
    fig = plot_path_rover_frame(arc, fig=fig)
    fig = plot_path_rover_frame(best_arc, color="green", fig=fig)
fig.show()
print(control)


# Arc image overlay visualization


In [ ]:
from lac.utils.camera import Camera
from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE

In [ ]:
def rover_pose_to_cam_pose(rover_pose, cam_name="FrontLeft"):
    camera_pose = get_cam_pose_rover(cam_name)
    camera_pose[:3, :3] = CAMERA_TO_OPENCV_PASSIVE
    return rover_pose @ camera_pose